# Capital Allocation Line #

### Building Portfolios Maximizing Sharpe Ratio ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data
from markowitz_portfolios_toolkit import portfolio_variance
from markowitz_portfolios_toolkit import eff_coefficients
from markowitz_portfolios_toolkit import eff_equation
from markowitz_portfolios_toolkit import markowitz_weights

In [2]:
tickers = get_tickers(mod="2.3")

tickers

In [3]:
# Import data
df_returns = pd.DataFrame()

for ticker in tickers:
    df = get_market_data(
        ticker=ticker, 
        start_date='2015-01-01', 
        end_date='2025-01-01', 
        returns=True
    )
    
    returns = df['returns'].rename(ticker)
    
    df_returns = pd.concat([df_returns, returns], axis=1)
    
    print(f'Data Ready for {ticker}')

In [4]:
df_returns

In [5]:
# Create the expected returns and standard deviations
mean_returns = df_returns.mean()
volatility = df_returns.dropna().std()
covariances = df_returns.dropna().cov()

In [6]:
# Get the coefficients of the Efficient Frontier
coefficients = eff_coefficients(mean_returns, covariances)

coefficients

In [7]:
# Create a rango of values for mu_P
mu_P_values = np.linspace(0.0, 0.004, 400)

# Evaluate the equation for mu_P values
sigma_P_values = eff_equation(coefficients, mu_P_values).reshape(-1, 1)

In [8]:
# Create the Plot
plt.figure(figsize=(10, 6))
plt.plot(sigma_P_values, mu_P_values, label=r'Efficient Frontier', color='black')


# Config
plt.title('Efficient Frontier')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [9]:
# The Most Efficient Portfolio is that which maximizes the Sharp Ratio
rfr = 0.0001

Let us find the Tangency Portfolio

Tangency Returns: $ \mu_T = \frac{2\pi_0 - \pi_1r_f}{\pi_1 - 2\pi_2r_f} $

In [10]:
# Get the values
pi_0 = coefficients[0]
pi_1 = coefficients[1]
pi_2 = coefficients[2]

t_returns = ((2*pi_0 - pi_1*rfr)/(pi_1 - 2*pi_2*rfr))
t_volatility = eff_equation(coefficients, t_returns)

print(f"The Tangency Portfolio Returns are: {t_returns}")
print(f"The Tangency Portfolio Volatility is: {t_volatility}")

In [11]:
# Create the Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(t_volatility, t_returns, color='red', s=50, label='Tangency Portfolio')  
plt.plot(sigma_P_values, mu_P_values, label=r'Efficient Frontier', color='black')


# Config
plt.title('Efficient Frontier and Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [12]:
# The Maximum Sharpe Ratio is the slope of the capital allocation line
sharpe = (t_returns - rfr)/t_volatility

print(f"The Maximum Sharpe Ratio is: {sharpe}")

In [13]:
# Define the CAL
def cal_equation(
        risk_free_rate,
        sharpe_ratio,
        sigma_P
):
    return risk_free_rate + sharpe_ratio * sigma_P

In [14]:
# Create Range for Sigma
sigma_cal_values = np.linspace(0.0, 0.08, 400)

# Evaluate for each value of sigma
mu_cal_values = cal_equation(rfr, sharpe, sigma_cal_values).reshape(-1, 1)

In [15]:
# Create Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(t_volatility, t_returns, color='red', s=50, label='Tangency Portfolio')  
plt.plot(sigma_P_values, mu_P_values, label=r'Efficient Frontier', color='black')
plt.plot(sigma_cal_values, mu_cal_values, label=r'Capital Allocation Line', color='black', linestyle='--')
plt.axhline(y=rfr, color='r', linestyle='--', label='Risk-Free Rate')

# Config
plt.title('Efficient Frontier and Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [16]:
# Get the Weights of the Tangency Portfolio
def weights(
        desired_returns,
        expected_returns,
        covariance_matrix
):
    # Number of assets
    n = len(expected_returns)
    
    # Create inputs
    mu = expected_returns.values.flatten().reshape(-1, 1)  # Expected Returns
    Sigma = covariance_matrix.values  # Covariance Matrix
    Sigma_inv = np.linalg.inv(Sigma)  # Inverse Covariance Matrix
    iota = np.ones((n, 1))  # Vector of Ones

    # Create components
    A = np.dot(np.dot(mu.T, Sigma_inv), mu)
    B = np.dot(np.dot(iota.T, Sigma_inv), mu)
    C = np.dot(np.dot(iota.T, Sigma_inv), iota)
    D = (A * C) - (B * B)

    # Calculate the weights
    first_part = (((desired_returns * C) - B) / D) * (Sigma_inv @ mu)
    second_part = ((A - (desired_returns * B)) / D) * (Sigma_inv @ iota)

    return first_part + second_part 

In [17]:
# Calculate the weights
tangency_weights = markowitz_weights(mean_returns, covariances, t_returns)

tangency_weights

In [18]:
# Now let us assume we there are an investor willing to take lower risk to reach worse returns
cal_returns = 0.0005

#In a normal case
normal_case_weights = markowitz_weights(mean_returns, covariances, cal_returns)

print(normal_case_weights)
print(f'The sum of weights is: {normal_case_weights.sum().round(2)}')

In [22]:
# Define the function to get the weights for the CAL
def capital_allocation_line_weights(
        desired_returns,
        tangency_returns,
        risk_free_rate,
):
    # Calculate Tangents Weights
    tan_ws = weights(tangency_returns, mean_returns, covariances)
    
    # Calculate discount factor
    discount_factor = (desired_returns - risk_free_rate) / (tangency_returns - risk_free_rate)
    
    # Calculate weights
    cal_weights = tan_ws * discount_factor

    return cal_weights

In [23]:
# Calculate the weights
cal_ws = capital_allocation_line_weights(cal_returns, t_returns, rfr)

print(cal_ws)
print(f'The sum of weights is: {cal_ws.sum().round(4)}')

if cal_ws.sum() < 1:
    print('You are a lender')
elif cal_ws.sum() > 1:
    print('You are a borrower')
else:
    print('You are special')

In [24]:
# Get the volatility given the desired returns
def capital_allocation_line_volatility(
        desired_returns,
        risk_free_rate,
        sharpe_ratio,
):
    # Calculate the volatility
    sigma = (desired_returns - risk_free_rate) / sharpe_ratio
    
    return sigma

In [26]:
# Calculate Volatility
cal_volatility = capital_allocation_line_volatility(cal_returns, rfr, sharpe)

print(f'The CAL Portfolio Risk: {cal_volatility}')

In [27]:
# You can get the same result by using the standard equation
cal_variance = portfolio_variance(cal_ws, df_returns)
cal_volatility_alt = np.sqrt(cal_variance)

print(f'CAL Portfolio Variance: {cal_volatility_alt[0][0]}')

In [28]:
# Create Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(t_volatility, t_returns, color='red', s=50, label='Tangency Portfolio')  
plt.plot(sigma_P_values, mu_P_values, label=r'Efficient Frontier', color='black')
plt.plot(sigma_cal_values, mu_cal_values, label=r'Capital Allocation Line', color='black', linestyle='--')
plt.scatter(cal_volatility, cal_returns, color='blue', s=50, label='CAL Portfolio')
plt.axhline(y=rfr, color='r', linestyle='--', label='Risk-Free Rate')


# Config
plt.title('Efficient Frontier and Portfolios')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [29]:
# Create Portfolios
tangency_portfolio = df_returns @ tangency_weights

# Create DataFrame
df_returns_ports = df_returns.copy()

df_returns_ports['Tangency Portfolio'] = tangency_portfolio

df_returns_ports

### Comparing Different Portfolios ###

In [31]:
# Define the desired portfolios
returns_list = [0.001, 0.0015, 0.0025, 0.003]

# Loop over desired returns with index
for r, ret in enumerate(returns_list):
    
    # Calculate Weights
    ws = capital_allocation_line_weights(ret, t_returns, rfr)
    
    # Calculate the Portfolio Returns
    portfolio = df_returns.values @ ws
    
    # Save it in the DataFrame
    df_returns_ports[f'port_{r}'] = portfolio
    

In [32]:
df_returns_ports

In [33]:
def calculate_analytics(returns_dataframe, risk_free_rate=0.0):
    # Trading Days in one Year
    ann_factor = 252  
    
    # Annualized Returns
    annualized_return = returns_dataframe.mean() * ann_factor
    
    # Annualized Volatility
    annualized_std = returns_dataframe.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_std
    
    # Max Drawdown
    cumulative_returns = (1 + returns_dataframe.div(100)).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns / rolling_max) - 1
    max_drawdown = drawdown.min()

    # VaR at 95%
    var_95 = returns_dataframe.quantile(0.05)

    # Create DF
    summary_df = pd.DataFrame({
        "Annualized Returns": annualized_return,
        "Annualized Volatility": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "VaR 95%": var_95
    })
    
    return summary_df

In [34]:
# Now the table
analytics_table = calculate_analytics(df_returns_ports)

analytics_table